In [1]:
from scipy import stats
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import gumbel_r
from scipy.stats import gumbel_l
import os
import os.path
import math

In [2]:
out_path = r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6'
csv_path = os.path.join(out_path,'csv_file','pr_day_2021-2060','All_projects_moz_precipitation_2021-2060_without_month_year.csv')
precipitation_2021_2060 = pd.read_csv(csv_path,header=[0,1],index_col=[0,1,2,3,4])*86400 # units kg.m^(-2).s^(-1)
# convert precipitation data from kg.m^(-2).s^(-1) to mm/day :  1 kg/m2/s = 86400 mm/day
# source: https://www.researchgate.net/post/How-do-I-convert-ERA-Interim-precipitation-estimates-from-kg-m2-s-to-mm-day
list_time_2021_2060=pd.date_range('01-01-2021','31-12-2060', freq='D').strftime('%d-%m-%Y').values.tolist()

In [3]:
precipitation_2021_2060_copy = precipitation_2021_2060.copy(deep=True) # make a coåy that can be modified without changin the original

In [4]:
precipitation_2021_2060_copy

Longitude  \
                                                                    33.625   
Name project          Experiment Model      Date       Latitude              
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125    5.057179   
                                            02-01-2021 -16.125   17.624178   
                                            03-01-2021 -16.125    7.632648   
                                            04-01-2021 -16.125    4.736492   
                                            05-01-2021 -16.125    4.981353   
...                                                                    ...   
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125         NaN   
                                            28-12-2060 -13.125         NaN   
                                            29-12-2060 -13.125         NaN   
                                            30-12-2060 -13.125         NaN   
                                            31-12-2060 -13.125         NaN   

                                                                               \
                                                                39.875 36.875   
Name project          Experiment Model      Date       Latitude                 
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125     NaN    NaN   
                                            02-01-2021 -16.125     NaN    NaN   
                                            03-01-2021 -16.125     NaN    NaN   
                                            04-01-2021 -16.125     NaN    NaN   
                                            05-01-2021 -16.125     NaN    NaN   
...                                                                ...    ...   
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125     NaN    NaN   
                                            28-12-2060 -13.125     NaN    NaN   
                                            29-12-2060 -13.125     NaN    NaN   
                                            30-12-2060 -13.125     NaN    NaN   
                                            31-12-2060 -13.125     NaN    NaN   

                                                                           
                                                                   40.375  
Name project          Experiment Model      Date       Latitude            
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125        NaN  
                                            02-01-2021 -16.125        NaN  
                                            03-01-2021 -16.125        NaN  
                                            04-01-2021 -16.125        NaN  
                                            05-01-2021 -16.125        NaN  
...                                                                   ...  
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125   0.000000  
                                            28-12-2060 -13.125   0.435942  
                                            29-12-2060 -13.125   6.300328  
                                            30-12-2060 -13.125   8.168522  
                                            31-12-2060 -13.125   0.737792  

[6136200 rows x 4 columns]

In [5]:
# delete model 'NESM3' that does not have any value
precipitation_2021_2060_copy = precipitation_2021_2060.copy(deep=True)
precipitation_2021_2060_copy.drop('NESM3', level=2, axis=0, inplace=True)
precipitation_2021_2060_copy.loc[(precipitation_2021_2060_copy.index.levels[0][0],precipitation_2021_2060_copy.index.levels[1][0],'NESM3')]

C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_191216\1858925233.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  precipitation_2021_2060_copy.loc[(precipitation_2021_2060_copy.index.levels[0][0],precipitation_2021_2060_copy.index.levels[1][0],'NESM3')]


KeyError: ('Chimoio_2ML_Reservoir', 'ssp126', 'NESM3')

In [ ]:
# treatment of dataframe to make useable
df_years=precipitation_2021_2060_copy.copy(deep=True)
#df_year = df_year1.loc[('PT_Revubue_2_Rev_2_01')] # select one project
df_years.columns = df_years.columns.droplevel(0) # drop first level of column name
df_years['33.625'] = df_years['33.625'].fillna(0)
df_years['39.875'] = df_years['39.875'].fillna(0)
df_years['36.875'] = df_years['36.875'].fillna(0)
df_years['40.375'] = df_years['40.375'].fillna(0)
df_years['33.625'] = df_years['33.625'] + df_years['39.875'] + df_years['36.875'] + df_years['40.375']
df_years = df_years[['33.625']]
df_years=df_years.rename(columns={df_years.columns[0]:'Yearly average Precipitation mm/day period 2021-2060'})
df_years=df_years.droplevel(level=4) # drop latitude index
df_years=df_years.reset_index()

Year = df_years[['Date']].values.reshape(len(df_years[['Date']].values),)
for i in np.arange(0,len(df_years[['Date']].values)):
    Year[i]=str(Year[i][6:10])

df_years['Year'] = Year
df_years=df_years.drop(labels='Date',axis=1)

df_years_mean = df_years.groupby(['Name project','Experiment','Model','Year']).mean() # maximum

In [ ]:
df_years_mean

In [ ]:
df_years_mean.groupby(['Name project']).describe(percentiles=[.1, .5, .9])
# if describe() does not return al wanted statistics, it is maybe because the elements in it are not recognized as int
# add astype(int) as in following example; df.astype(int).groupby(['Name project']).describe(percentiles=[.1, .5, .9])

### Seasonal average precipitation

In mozambique, dry season from April to september. Humid season from October to March

In [ ]:
# deal with weird organisation
df_season = precipitation_2021_2060_copy.copy(deep=True)
df_season.columns = df_season.columns.droplevel(0) # drop first level of column name
df_season['33.625'] = df_season['33.625'].fillna(0)
df_season['39.875'] = df_season['39.875'].fillna(0)
df_season['36.875'] = df_season['36.875'].fillna(0)
df_season['40.375'] = df_season['40.375'].fillna(0)
df_season['33.625'] = df_season['33.625'] + df_season['39.875'] + df_season['36.875'] + df_season['40.375']
df_season = df_season[['33.625']]
df_season=df_season.rename(columns={df_season.columns[0]:'Yearly average precipitation mm/day period 2021-2060'})
df_season=df_season.droplevel(level=4) # drop latitude index
df_season=df_season.reset_index()

Month = df_season[['Date']].values.reshape(len(df_season[['Date']].values),)
Season = df_season[['Date']].values.reshape(len(df_season[['Date']].values),)
for i in np.arange(0,len(df_season[['Date']].values)):
    Month[i]=Month[i][3:5]
    if int(Month[i])>3 and int(Month[i])<10:
        Season[i]='Dry'
    else:
        Season[i]='Humid'
    

#df_season['Month'] = Month
df_season['Season'] = Season
df_season=df_season.drop(labels='Date',axis=1)
df_season

In [ ]:
df_season_mean=df_season.groupby(['Name project','Experiment','Model','Season']).mean()
df_season_mean

In [ ]:
df_season_mean.groupby(['Name project','Season']).describe(percentiles=[.1, .5, .9])

# Distinguish 2 periods of time

In [11]:
df_season_2 = precipitation_2021_2060_copy.copy(deep=True)
df_season_2

Longitude  \
                                                                    33.625   
Name project          Experiment Model      Date       Latitude              
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125    5.057179   
                                            02-01-2021 -16.125   17.624178   
                                            03-01-2021 -16.125    7.632648   
                                            04-01-2021 -16.125    4.736492   
                                            05-01-2021 -16.125    4.981353   
...                                                                    ...   
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125         NaN   
                                            28-12-2060 -13.125         NaN   
                                            29-12-2060 -13.125         NaN   
                                            30-12-2060 -13.125         NaN   
                                            31-12-2060 -13.125         NaN   

                                                                               \
                                                                39.875 36.875   
Name project          Experiment Model      Date       Latitude                 
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125     NaN    NaN   
                                            02-01-2021 -16.125     NaN    NaN   
                                            03-01-2021 -16.125     NaN    NaN   
                                            04-01-2021 -16.125     NaN    NaN   
                                            05-01-2021 -16.125     NaN    NaN   
...                                                                ...    ...   
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125     NaN    NaN   
                                            28-12-2060 -13.125     NaN    NaN   
                                            29-12-2060 -13.125     NaN    NaN   
                                            30-12-2060 -13.125     NaN    NaN   
                                            31-12-2060 -13.125     NaN    NaN   

                                                                           
                                                                   40.375  
Name project          Experiment Model      Date       Latitude            
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125        NaN  
                                            02-01-2021 -16.125        NaN  
                                            03-01-2021 -16.125        NaN  
                                            04-01-2021 -16.125        NaN  
                                            05-01-2021 -16.125        NaN  
...                                                                   ...  
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125   0.000000  
                                            28-12-2060 -13.125   0.435942  
                                            29-12-2060 -13.125   6.300328  
                                            30-12-2060 -13.125   8.168522  
                                            31-12-2060 -13.125   0.737792  

[5727120 rows x 4 columns]

In [12]:
## PROBLEM
df_season_2.columns = df_season_2.columns.droplevel(0) # drop first level of column name
df_season_2['33.625'] = df_season_2['33.625'].fillna(0)
df_season_2['39.875'] = df_season_2['39.875'].fillna(0)
df_season_2['36.875'] = df_season_2['36.875'].fillna(0)
df_season_2['40.375'] = df_season_2['40.375'].fillna(0)
df_season_2['33.625'] = df_season_2['33.625'] + df_season_2['39.875'] + df_season_2['36.875'] + df_season_2['40.375']
df_season_2 = df_season_2[['33.625']]
df_season_2=df_season_2.rename(columns={df_season_2.columns[0]:'Yearly average precipitation mm/day period 2021-2060'})
df_season_2=df_season_2.droplevel(level=4) # drop latitude index
df_season_2=df_season_2.reset_index()

Year = df_season_2[['Date']].values.reshape(len(df_season_2[['Date']].values),)
for i in np.arange(0,len(df_season_2[['Date']].values)):
    Year[i]=Year[i][6:10]

#df_season_2['Year'] = Year
#df_season_2=df_season_2.drop(labels='Date',axis=1)

In [13]:
df_season_2

,Name project,Experiment,Model,Date,Yearly average precipitation mm/day period 2021-2060
0,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,01-01-2021,5.057179
1,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,02-01-2021,17.624178
2,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,03-01-2021,7.632648
3,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,04-01-2021,4.736492
4,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,05-01-2021,4.981353
...,...,...,...,...,...
5727115,PT__Dumping_Site,ssp370,TaiESM1,27-12-2060,0.000000
5727116,PT__Dumping_Site,ssp370,TaiESM1,28-12-2060,0.435942
5727117,PT__Dumping_Site,ssp370,TaiESM1,29-12-2060,6.300328
5727118,PT__Dumping_Site,ssp370,TaiESM1,30-12-2060,8.168522


In [14]:
df_season_2['Year'] = Year
df_season_2

,Name project,Experiment,Model,Date,Yearly average precipitation mm/day period 2021-2060,Year
0,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,01-01-2021,5.057179,2021
1,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,02-01-2021,17.624178,2021
2,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,03-01-2021,7.632648,2021
3,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,04-01-2021,4.736492,2021
4,PT_Revubue_2_Rev_2_01,ssp245,ACCESS-CM2,05-01-2021,4.981353,2021
...,...,...,...,...,...,...
5727115,PT__Dumping_Site,ssp370,TaiESM1,27-12-2060,0.000000,2060
5727116,PT__Dumping_Site,ssp370,TaiESM1,28-12-2060,0.435942,2060
5727117,PT__Dumping_Site,ssp370,TaiESM1,29-12-2060,6.300328,2060
5727118,PT__Dumping_Site,ssp370,TaiESM1,30-12-2060,8.168522,2060


In [ ]:
filtered_df_season_2 = df_season_2[df_season_2['Date'].str.contains('2021','2022')]
filtered_df_season_2

In [ ]:
df_season_2.index

In [17]:
df_season_2['Year'].between(2024,2030)

TypeError: '>=' not supported between instances of 'str' and 'int'

In [15]:
df_season_2.loc[df_season_2['Year'].between(2021,2030)]

TypeError: '>=' not supported between instances of 'str' and 'int'